In [1]:
import os
import network
import torch
from sklearn.metrics import confusion_matrix
import numpy as np
from data_list import ImageList, ImageList_idx, ImageList_idx_aug, ImageList_idx_aug_fix
from torch.utils.data import DataLoader
from torchvision import transforms
import argparse
import time

In [2]:
def image_test(resize_size=256, crop_size=224, alexnet=False):
    ## transform
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  #else:
    #normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        normalize
    ])

In [3]:
def data_load(t_dset_path,batch_size): 
    ## prepare data
    dsets = {}
    dset_loaders = {}
    train_bs = batch_size
    txt_tar = open(t_dset_path).readlines()
    dsets["test"] = ImageList_idx(txt_tar, transform=image_test())
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=train_bs*3, shuffle=False, num_workers=4, drop_last=False)
    return dset_loaders

In [4]:
def net_load(backbone,class_num,bottleneck_dim,path_F,path_B,path_C):
    ## load network
    netF = network.ResBase(res_name=backbone).cuda()
    netB = network.feat_bootleneck(type='bn', feature_dim=netF.in_features, bottleneck_dim=bottleneck_dim).cuda()
    netC = network.feat_classifier(type='wn', class_num = class_num, bottleneck_dim=bottleneck_dim).cuda()
    modelpath = path_F   
    netF.load_state_dict(torch.load(modelpath))
    modelpath = path_B
    netB.load_state_dict(torch.load(modelpath))
    modelpath = path_C   
    netC.load_state_dict(torch.load(modelpath))
    return netF,netB,netC

In [5]:
def cal_acc(loader, netF, netB, netC):
    ## calculate accuracy
    start_test = True
    with torch.no_grad():
        iter_test = iter(loader)
        for i in range(len(loader)):
            data = next(iter_test)
            inputs = data[0]
            labels = data[1]
            inputs = inputs.cuda()
            outputs = netC(netB(netF(inputs)))
            if start_test:
                all_output = outputs.float().cpu()
                all_label = labels.float()
                start_test = False
            else:
                all_output = torch.cat((all_output, outputs.float().cpu()), 0)
                all_label = torch.cat((all_label, labels.float()), 0)
    _, predict = torch.max(all_output, 1)
    accuracy = torch.sum(torch.squeeze(predict).float() == all_label).item() / float(all_label.size()[0])
    return accuracy*100

In [6]:
path_f = './Demo-model-transfer-C2A/target_F_par_0.6.pt'
path_b = './Demo-model-transfer-C2A/target_B_par_0.6.pt'
path_c = './Demo-model-transfer-C2A/target_C_par_0.6.pt'
netF,netB,netC = net_load('resnet50',65,512,path_f,path_b,path_c)
path_target = './data/office-home/Art_list.txt'
dset_loaders = data_load(path_target,64)
netF.eval()
netB.eval()
netC.eval()

/home/imi/anaconda3/envs/sfa/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/imi/anaconda3/envs/sfa/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


feat_classifier(
  (fc): Linear(in_features=512, out_features=65, bias=True)
)

In [7]:
acc_s_te = cal_acc(dset_loaders['test'], netF, netB, netC)
acc_s_te

79.23362175525341